# How to Render a Metal Design into Ansys

## Prerequisite
You need to have Ansys installed locally (any version) - at the time of this notebook, Ansys is only supported in Windows.

## 1. Create the design in Metal
#### 1.1 Preload libraries and classes, then open Metal
Let's start by importing all the key libraries and classes, and opening Metal

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import qiskit_metal as metal
from qiskit_metal import designs, MetalGUI
from qiskit_metal import Dict, Headings

In [14]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer
QAnsysRenderer.default_options

{'Lj': '10nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'ansys_file_extension': '.aedt',
 'x_buffer_width_mm': 0.2,
 'y_buffer_width_mm': 0.2,
 'plot_ansys_fields_options': {'name': 'NAME:Mag_E1',
  'UserSpecifyName': '0',
  'UserSpecifyFolder': '0',
  'QuantityName': 'Mag_E',
  'PlotFolder': 'E Field',
  'StreamlinePlot': 'False',
  'AdjacentSidePlot': 'False',
  'FullModelPlot': 'False',
  'IntrinsicVar': "Phase='0deg'",
  'PlotGeomInfo_0': '1',
  'PlotGeomInfo_1': 'Surface',
  'PlotGeomInfo_2': 'FacesList',
  'PlotGeomInfo_3': '1'}}

In [15]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

In [16]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander

#### 1.2 Prepare a design consisting of 4 qubits and 4 CPWs
First we import the necessary components and change key global variables to the desired size.

In [10]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander

In [7]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

In [17]:
RouteMeander.get_template_options(design)

{'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'chip': 'main',
 'layer': '1',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False}

Let's create the 4-Qubit-4-CPWs design in one shot. Feel free to break the cell below into subcells if you would like to more closely follow the layout generation.

In [18]:
# Enable overwrite, so that we can re-un this cell multiple times without incurring object name collisions
design.overwrite_enabled = True

# We now define the same custom options for all the transmons.
# We only list the options we intend to modify with respect to the default values
options = Dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 3 connection_pads
    connection_pads=Dict(
        a = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

# Create the 4 transmons
q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

# Now we do the same for the CPWs
options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

# We define a handy method to shorten the number of line of code necessary to define the 4 CPW
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(hfss_wire_bonds=True,
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Create the 4 CPWs
asym = 140
cpw1 = connect('cpw1', 'Q1', 'c', 'Q2', 'b', '5.6 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q2', 'c', '5.7 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'c', 'Q4', 'b', '5.6 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'c', '5.7 mm', f'-{asym}um', flip=True)

# Render the entire design to the Qiskit Metal GUI
gui.rebuild()
gui.autoscale()

## 2. Render into Ansys HFSS
The Ansys HFSS renderer was instantiated during the boot of Qiskit Metal. Let's create a short handle to refer to it in the rest of this notebook.

In [9]:
fourq_hfss = design.renderers.hfss

#### 2.1 Setup an Ansys project
To setup the project **manually**, follow these instructions:
1. Launch `ANSYS Electronics Desktop yyyy Rx` (from your Windows Start menu).
2. Create a new Ansys project by clicking on the `New` icon at the top left. (or open an existing project)

Alternatively, you can **automatically** set up the project by executing the following two cells. Make sure to wait after executing the first cell for Ansys to completely open. Only then execute the second cell.

Note about Ansys version: open_ansys() will look by default for the 2020 R2 version of Ansys. You can easily reroute it to your Ansys of choice by providing the name of the environment variable that contains the path (path_env) or the path itself (path)

In [ ]:
fourq_hfss.open_ansys()   # this opens Ansys 2020 R2 if present
# fourq_hfss.open_ansys(path_var='ANSYSEM_ROOT211')
# fourq_hfss.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM20.2\Win64')
# fourq_hfss.open_ansys(path='../../../Program Files/AnsysEM/AnsysEM20.2/Win64')

NOTE: A new project should have automatically opened with the execution of the cell above. If not, uncomment the cell below and execute it. You can also load an existing project by passing the project information to the `connect_ansys()`.

In [ ]:
# fourq_hfss.new_ansys_project()

#### 2.2 Connect the Metal renderer with the Ansys project
Open either a new or existing design based on default options

In [10]:
fourq_hfss.connect_ansys()
# fourq_hfss.connect_ansys('C:\\project_path\\', 'Project1')  # Example of opening a saved project

INFO 01:17PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:17PM [load_ansys_project]: 	Opened Ansys App
INFO 01:17PM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 01:17PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/ThomasMcConkey/Documents/Ansoft/
	Project:   Project7
INFO 01:17PM [connect_design]: 	Opened active design
	Design:    HFSSDesign1 [Solution type: DrivenModal]
INFO 01:17PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 01:17PM [connect]: 	Connected to project "Project7" and design "HFSSDesign1" 😀 



You can also indicate, using the optional parameters, whether you intend to open and use a previously saved project. <br>
Make sure that the saved project contains at least one design, or the method will produce a warning

#### 2.3 Setup an Ansys HFSS design
You can either create a new design or select and use an old one.

To **create** a new design **manually**, go to the Ansys GUI and follow these instructions:
1. Select the project from the leftmost menu in the Ansys GUI.
2. Go into the menu `Project` and select `Insert HFSS Design`.
3. Change the HFSS design to either eigenmode or modal by right-clicking on the HFSSdesign1 that just got created inside your project (left panel) and then selecting: `Solution Type...`.

To **create** a new eigenmode design **automatically**, execute the following cell<br>
The design will be added to the project that was active when the command `fourq_hfss.connect_ansys()` was executed.
Note: If a design named `HFSSTransmonQubit` already exists in the project, a new design will be created, with the name suffixed with an incremental integer: `HFSSTransmonQubit1`, `HFSSTransmonQubit2`, etc.

In [ ]:
fourq_hfss.add_eigenmode_design("HFSSMetalEigenmode")

To **create** a new modal design **automatically**, execute the following cell instead

In [11]:
fourq_hfss.add_drivenmodal_design("HFSSMetalDrivenModal")

INFO 01:17PM [connect_design]: 	Opened active design
	Design:    HFSSMetalDrivenModal [Solution type: DrivenModal]
WARNING 01:17PM [connect_setup]: 	No design setup detected.
WARNING 01:17PM [connect_setup]: 	Creating drivenmodal default setup.
INFO 01:17PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)


In [21]:
fourq_hfss.modeler.draw_wirebond

<bound method HfssModeler.draw_wirebond of <pyEPR.ansys.HfssModeler object at 0x00000163276E9748>>

In [33]:
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [30]:
design.qgeometry.tables['path']

,component,name,geometry,layer,subtract,helper,chip,width,fillet,hfss_wire_bonds,q3d_wire_bonds,gds_wire_bonds
0,1,a_wire,"LINESTRING (2.63501 -0.13000, 2.66001 -0.13000...",1,False,False,main,0.010,NaN,False,False,NaN
1,1,a_wire_sub,"LINESTRING (2.63501 -0.13000, 2.66001 -0.13000...",1,True,False,main,0.022,NaN,False,False,NaN
2,1,b_wire,"LINESTRING (2.21001 0.13000, 2.18501 0.13000, ...",1,False,False,main,0.010,NaN,False,False,NaN
3,1,b_wire_sub,"LINESTRING (2.21001 0.13000, 2.18501 0.13000, ...",1,True,False,main,0.022,NaN,False,False,NaN
4,1,c_wire,"LINESTRING (2.21001 -0.13000, 2.18501 -0.13000...",1,False,False,main,0.010,NaN,False,False,NaN
5,1,c_wire_sub,"LINESTRING (2.21001 -0.13000, 2.18501 -0.13000...",1,True,False,main,0.022,NaN,False,False,NaN
6,2,a_wire,"LINESTRING (-0.13000 -1.16250, -0.13000 -1.187...",1,False,False,main,0.010,NaN,False,False,NaN
7,2,a_wire_sub,"LINESTRING (-0.13000 -1.16250, -0.13000 -1.187...",1,True,False,main,0.022,NaN,False,False,NaN
8,2,b_wire,"LINESTRING (0.13000 -0.73750, 0.13000 -0.71250...",1,False,False,main,0.010,NaN,False,False,NaN
9,2,b_wire_sub,"LINESTRING (0.13000 -0.73750, 0.13000 -0.71250...",1,True,False,main,0.022,NaN,False,False,NaN


To **select** an existing design, you can follow one of three ways:
* re-running the `fourq_hfss.connect_ansys(*with parameters*)`, this time specifying which design to connect to (see section 2.2)
* manually activating the design from the Ansys GUI. You will find the list of designs in the leftmost panel, and you can activate them with a double click. After this, re-run the `fourq_hfss.connect_ansys()` *without parameters*.
* use methods: activate_eigenmode_design() or activate_drivenmodal_design().  If the design name exists, it will be added, but no integer will be added to the suffix. If the design name does not exist, then it will be added to the project.

Let's illustrate the **automated** method to select an existing design. The following two cells will switch between the eigenmode and the modal designs we just created in the last two cells above, or it will create them if you did not run the previous cells:

In [ ]:
fourq_hfss.activate_eigenmode_design("HFSSMetalEigenmode")

In [ ]:
fourq_hfss.activate_drivenmodal_design("HFSSMetalDrivenModal")

#### 2.4 Render some component from the Metal design

Find below several rendering examples. You can choose to only execute one of them if you are just browsing this notebook.

Notice how we explicitly clear the design before re-rendering. Indeed `render_design()` only adds shapes to the Ansys design. Re-rendering the same shapes will cause violations.

In [12]:
fourq_hfss.render_design([], [])  # entire Metal design.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')])  # single qubit with 2 endcaps.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')])  # 2 qubits and 2 endcaps, one per qubit.

For Driven-Modal analysis, we can also add terminations. In the example below we render 1 qubit with 1 endcap and 1 port with a 70 Ohm termination.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design(['Q2'], [('Q2', 'a')], [('Q2', 'b', '70')])

In the previous examples, rendering area dimenstions is determined by the size of the selected geometries, with some buffer.

For a more accurate control of the chip size, you need to disable the buffering as below. This will use `design._chips['main']['size']` to determine the rendering area dimensions.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design([], [], box_plus_buffer=False)

You can also modify the chip size directly by updating `design._chips['main']['size']`. Example below.

NOTE: we purposfully make the chip size smaller than the size of the geometry. This will cause a warning to show which will need to be fixed by the user intending to conduct a valid analysis.

In [ ]:
fourq_hfss.clean_active_design()
design._chips['main']['size']['size_x'] = '4mm'
fourq_hfss.render_design([], [], box_plus_buffer=False)
#    Return back to original size, for the remainder of the notebook
design._chips['main']['size']['size_x'] = '6mm'

#### 2.5 Create of select a setup
This section is only needed if you plan to execute an analysis. This topic will be covered in the tutorials in chapter 6.

To **select** a setup for the active design, you can use `activate_drivenmodal_setup()` or `activate_eigenmode_setup()`. If the setup exists, the QRenderer will reference the setup, otherwise, will make a new setup with the name give.  If no name given, the default name of "Setup" will be used. Below the example for eigenmode.

In [ ]:
fourq_hfss.activate_eigenmode_setup('SetupNEW')

**Finally** disconnect the Metal renderer from the Ansys session.

NOTE: This is needed every time before re-connecting. If you do not disconnect explicitly, you might not be able to close the Ansys GUI later.

In [25]:
fourq_hfss.disconnect_ansys()

## 3. Render into Ansys Q3D
The Ansys Q3D renderer was instantiated during the boot of Qiskit Metal. Let's create a short handle to refer to it in the rest of this notebook.

In [ ]:
fourq_q3d = design.renderers.q3d

#### 3.1 Setup an Ansys project
Skip this section if ansys is already open. See section 2.1 in this notebook for additional details and options.

In [ ]:
fourq_q3d.open_ansys()

#### 3.2 Connect the Q3D renderer with the Ansys project
Open either a new or existing design based on default options. See section 2.2 in this notebook for details and options.

In [ ]:
fourq_q3d.connect_ansys()
# fourq_q3d.connect_ansys('C:\\project_path\\', 'Project1')  # Example of opening a saved project

#### 3.3 Setup an Ansys Q3D design
You can either create a new design or select and use an old one.

To **create** a new design **manually**, go to the Ansys GUI and follow these instructions:
1. Select the project from the leftmost menu in the Ansys GUI.
2. Go into the menu `Project` and select `Insert Q3D Extractor Design`.

To **create** a new Q3D design **automatically**, execute the following cell.<br>
Note: If a design named `Q3dMetalDesign` already exists in the project, a new design will be created, with the name suffixed with an incremental integer: `Q3dMetalDesign1`, `Q3dMetalDesign2`, etc.

In [ ]:
fourq_q3d.add_q3d_design("Q3dMetalDesign")

To **select** an existing design, means to activate an Ansys design and connect to it. You can follow one of three ways:
* re-running the `fourq_q3d.connect_ansys(*with parameters*)`, this time specifying which design to connect to (see section 2.3)
* manually activating the design from the Ansys GUI. You will find the list of designs in the leftmost panel, and you can activate them with a double click. After this, re-run the `fourq_q3d.connect_ansys()` *without parameters*.
* use method: activate_q3d_design().  If the design name exists, it will be added, but no integer will be added to the suffix. If the design name does not exist, then it will be added to the project.

Let's illustrate the **automated** method to select an existing design. The first of the two cells below will create a new design because the name does not corrispond to any design in the project. The second cell will return to the previously created design.

In [ ]:
fourq_q3d.activate_q3d_design("Q3dMetalDesignNEW")

In [ ]:
fourq_q3d.activate_q3d_design("Q3dMetalDesign")

#### 3.4 Render some component from the Metal design

Find below several rendering examples. You can choose to only execute one of them if you are just browsing this notebook.

Notice how we explicitly clear the design before re-rendering. Indeed `render_design()` only adds shapes to the Ansys design. Re-rendering the same shapes will cause violations.

In [ ]:
fourq_q3d.render_design([], [])  # entire Metal design.

In [ ]:
fourq_q3d.clean_active_design()
fourq_q3d.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')])  # single qubit with 2 endcaps.

In [ ]:
fourq_q3d.clean_active_design()
fourq_q3d.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')])  # 2 qubits and 2 endcaps, one per qubit.

In the previous examples, rendering area dimenstions is determined by the size of the selected geometries, with some padding.

For a more accurate control of the chip size, you need to disable the buffering as below. This will use `design._chips['main']['size']` to determine the rendering area dimensions.

In [ ]:
fourq_q3d.clean_active_design()
fourq_q3d.render_design([], [], box_plus_buffer=False)

You can also modify the chip size directly by updating `design._chips['main']['size']`. Example below:

In [ ]:
fourq_q3d.clean_active_design()
design._chips['main']['size']['size_y'] = '4mm'
fourq_q3d.render_design([], [], box_plus_buffer=False)
#    Return back to original size, for the remainder of the notebook
design._chips['main']['size']['size_y'] = '6mm'

#### 3.5 Create of select a setup
This section is only needed if you plan to execute an analysis. This topic will be covered in the tutorials in chapter 6.

To **select** a setup for the active design, you can use `activate_q3d_setup()`. If the setup exists, the QRenderer will reference the setup, otherwise, will make a new setup with the name give.  If no name given, the default name of "Setup" will be used.

In [ ]:
fourq_q3d.activate_q3d_setup("SetupNEW")

**Finally** disconnect the Metal renderer from the Ansys session. You will not be able to close Ansys without executing this.

In [24]:
fourq_q3d.disconnect_ansys()

[autoreload of qiskit_metal.renderers.renderer_ansys.ansys_renderer failed: Traceback (most recent call last):
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

NameError: name 'fourq_q3d' is not defined

If you want to close the Metal GUI, uncomment the following cell

In [ ]:
# gui.main_window.close()

## References - Miscellaneous pyEPR/Ansys commands
The following commands are for reference only to better understand how the backend code works. They're not meant to be run directly in this notebook as part of the tutorial.

import pyEPR as epr

Connect to Ansys directly from notebook:

pinfo = epr.ProjectInfo(project_path = None, 
                        project_name = None,
                        design_name  = None)
modeler = pinfo.design.modeler

Access methods within HfssDesign class in pyEPR:

epr.ansys.HfssDesign.create_dm_setup
epr.ansys.HfssDesign.create_q3d_setup

Get project and design names:

pinfo.project_name
design._design.GetName()

Filter qgeometry table:

full_table = design.qgeometry.tables['poly']
mask = full_table['subtract'] == False
table = full_table[mask]

Draw centered rectangles:

bigsquare = modeler.draw_rect_center([0, 0, 0], x_size=8, y_size=8, name='bigsquare')
topright = modeler.draw_rect_center([2, 2, 0], x_size=2, y_size=2, name='topright')

Subtracting shapes:

modeler.subtract('bigsquare', ['topright'])

Draw centered box:

modeler.draw_box_center([0, 0, 0], [1, 2, 3])

Draw closed polygon:

trianglepts = [[-1, 5, 0], [1, 5, 0], [0, 7, 0]]
modeler.draw_polyline(trianglepts, closed=True)

Draw polyline:

smallpts = [[2.85, 0, 0], [3.15, 0, 0]]
modeler.draw_polyline(smallpts, closed=False)

Sweep one polyline with another:

modeler._sweep_along_path('Polyline8', 'Polyline7')